In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

In [2]:
#Dataset from prior investigation - NBA Combined Player Stats
#website - "https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20"

ps_player = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/NBA_Combined_Player_Stats.csv', index_col= 0)

In [3]:
ps_player.head()

,PlayerName,PlayerTeam,OpposingTeam,Date,GameID,Season,Min,SST,Pts,PPP,...,2FGM,2FGm,3FGA,3FGM,3FGm,FTA,FTM,FTm,PFTkn,PFCom
0,PJ Tucker,Houston Rockets,Philadelphia 76ers,2020-08-14,732811,2019 - 2020,29:03,0.53,7,1.17,...,2,0,4,1,3,0,0,0,1,2
1,Hamidou Diallo,Oklahoma City Thunder,Los Angeles Clippers,2020-08-14,732813,2019 - 2020,39:47,1.36,27,1.04,...,6,8,8,4,4,5,3,2,5,5
2,Danilo Galinari,Oklahoma City Thunder,Los Angeles Clippers,2020-08-14,732813,2019 - 2020,8:52,1.39,3,0.75,...,0,2,1,0,1,3,3,0,1,2
3,Nerlens Noel,Oklahoma City Thunder,Los Angeles Clippers,2020-08-14,732813,2019 - 2020,15:22,0.59,5,1.00,...,2,0,0,0,0,2,1,1,1,3
4,Steven Adams,Oklahoma City Thunder,Los Angeles Clippers,2020-08-14,732813,2019 - 2020,6:22,0.35,0,0.00,...,0,2,0,0,0,0,0,0,0,0


In [4]:
ps_player.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PlayerName    10 non-null     object 
 1   PlayerTeam    10 non-null     object 
 2   OpposingTeam  10 non-null     object 
 3   Date          10 non-null     object 
 4   GameID        10 non-null     int64  
 5   Season        10 non-null     object 
 6   Min           10 non-null     object 
 7   SST           10 non-null     float64
 8   Pts           10 non-null     int64  
 9   PPP           10 non-null     float64
 10  Ast           10 non-null     int64  
 11  T/O           10 non-null     int64  
 12  Ast/TO        10 non-null     float64
 13  Stl           10 non-null     int64  
 14  StlPos        10 non-null     int64  
 15  Blk           10 non-null     int64  
 16  BlkFGA        10 non-null     int64  
 17  TtlReb        10 non-null     int64  
 18  OffReb        10 non-null     int

In [5]:
#Dataset from prior investigation - NBA Combined Team Stats 
#website - "https://towardsdatascience.com/predicting-the-outcome-of-nba-games-with-machine-learning-a810bb768f20"

ps_team = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/NBA_Combined_Team_Stats.csv', index_col=0)

In [6]:
ps_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Columns: 199 entries, Date to A_3FG_Missed_4
dtypes: int64(191), object(8)
memory usage: 15.6+ KB


In [7]:
ps_team.head()

,Date,Game_ID,Season,H_Team,A_Team,H_Team_Record,A_Team_Record,H_Team_Home_Record,A_Team_Away_Record,H_Points,...,H_2FG_Made_4,A_2FG_Made_4,H_2FG_Missed_4,A_2FG_Missed_4,H_3FG_Attempts_4,A_3FG_Attempts_4,H_3FG_Made_4,A_3FG_Made_4,H_3FG_Missed_4,A_3FG_Missed_4
0,2020-08-14,732811,2019 - 2020,Houston Rockets,Philadelphia 76ers,44 - 27,42 - 30,24 - 11,11 - 26,96,...,5,7,3,2,13,11,2,3,11,8
1,2020-08-14,732814,2019 - 2020,Toronto Raptors,Denver Nuggets,52 - 19,46 - 26,25 - 10,20 - 15,117,...,7,6,7,9,10,6,6,4,4,2
2,2020-08-14,732812,2019 - 2020,Indiana Pacers,Miami Heat,44 - 28,44 - 28,24 - 11,15 - 21,109,...,8,5,7,8,10,8,3,2,7,6
3,2020-08-13,732808,2019 - 2020,Orlando Magic,New Orleans Pelicans,32 - 40,30 - 41,17 - 17,15 - 20,133,...,8,12,3,4,5,11,2,2,3,9
4,2020-08-13,732809,2019 - 2020,Phoenix Suns,Dallas Mavericks,33 - 39,43 - 31,16 - 22,23 - 13,128,...,5,7,7,8,9,6,3,1,6,5


In [9]:
#website - "https://www.kaggle.com/heeebsinc/nbaseasonstats201820"
#The above datasets are split into season from 2000 to 2020. We are only interested in 2008 - 2020.
cs_player= pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/2008-2009.csv')

In [10]:
cs_player.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30566 entries, 0 to 30565
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    30566 non-null  object 
 1   Date      30566 non-null  object 
 2   Team      30566 non-null  object 
 3   Against   30566 non-null  object 
 4   Home      30566 non-null  int64  
 5   MP        25638 non-null  float64
 6   FG        25638 non-null  float64
 7   FGA       25638 non-null  float64
 8   FG%       24286 non-null  float64
 9   3P        25638 non-null  float64
 10  3PA       25638 non-null  float64
 11  3P%       14274 non-null  float64
 12  FT        25638 non-null  float64
 13  FTA       25638 non-null  float64
 14  FT%       15393 non-null  float64
 15  ORB       25638 non-null  float64
 16  DRB       25638 non-null  float64
 17  TRB       25638 non-null  float64
 18  AST       25638 non-null  float64
 19  STL       25638 non-null  float64
 20  BLK       25638 non-null  fl

In [11]:
cs_player.head()

,Player,Date,Team,Against,Home,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,GameLink
0,LeBron James,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,36.00,9.0,21.0,0.429,0.0,...,6.0,7.0,6.0,2.0,1.0,3.0,4.0,22.0,-3.0,https://www.basketball-reference.com/boxscores...
1,Delonte West,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,35.29,2.0,4.0,0.500,0.0,...,2.0,2.0,2.0,0.0,0.0,1.0,0.0,6.0,-10.0,https://www.basketball-reference.com/boxscores...
2,Mo Williams,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,31.18,4.0,10.0,0.400,3.0,...,3.0,3.0,2.0,1.0,0.0,4.0,5.0,12.0,3.0,https://www.basketball-reference.com/boxscores...
3,Zydrunas Ilgauskas,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,30.24,3.0,5.0,0.600,0.0,...,5.0,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,https://www.basketball-reference.com/boxscores...
4,Ben Wallace,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,19.03,1.0,1.0,1.000,0.0,...,4.0,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,https://www.basketball-reference.com/boxscores...


In [12]:
print(cs_player['GameLink'])

0        https://www.basketball-reference.com/boxscores...
1        https://www.basketball-reference.com/boxscores...
2        https://www.basketball-reference.com/boxscores...
3        https://www.basketball-reference.com/boxscores...
4        https://www.basketball-reference.com/boxscores...
                               ...                        
30561    https://www.basketball-reference.com/boxscores...
30562    https://www.basketball-reference.com/boxscores...
30563    https://www.basketball-reference.com/boxscores...
30564    https://www.basketball-reference.com/boxscores...
30565    https://www.basketball-reference.com/boxscores...
Name: GameLink, Length: 30566, dtype: object


In [13]:
#Code for removing columns for future reference. 
#cs_player = cs_player.drop(['GameLink'], axis=1)

In [14]:
ps_player[['PlayerName','2FGA','2FGM','2FGm']][:2]

,PlayerName,2FGA,2FGM,2FGm
0,PJ Tucker,2,2,0
1,Hamidou Diallo,14,6,8


In [15]:
print(set(cs_player['Home']))

{0, 1}


In [16]:
#Question, when trying to add seasons to the dataset, it seems to immediately jump to 2019 instead of following
#the start year as noted below.
start_year = 2008
end_year = 2009

player_stats_combined_df = pd.DataFrame()

for i in range(12) :
  player_stats_file_path = '/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/player_stats/' + str(start_year) + "-" + str(end_year) + '.csv'

  player_stats_combined_df = player_stats_combined_df.append(pd.read_csv(player_stats_file_path))

  start_year = start_year + 1
  end_year = end_year + 1

In [17]:
player_stats_combined_df.head()

,Player,Date,Team,Against,Home,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,GameLink
0,LeBron James,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,36.00,9.0,21.0,0.429,0.0,...,6.0,7.0,6.0,2.0,1.0,3.0,4.0,22.0,-3.0,https://www.basketball-reference.com/boxscores...
1,Delonte West,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,35.29,2.0,4.0,0.500,0.0,...,2.0,2.0,2.0,0.0,0.0,1.0,0.0,6.0,-10.0,https://www.basketball-reference.com/boxscores...
2,Mo Williams,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,31.18,4.0,10.0,0.400,3.0,...,3.0,3.0,2.0,1.0,0.0,4.0,5.0,12.0,3.0,https://www.basketball-reference.com/boxscores...
3,Zydrunas Ilgauskas,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,30.24,3.0,5.0,0.600,0.0,...,5.0,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,https://www.basketball-reference.com/boxscores...
4,Ben Wallace,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,19.03,1.0,1.0,1.000,0.0,...,4.0,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,https://www.basketball-reference.com/boxscores...


In [18]:
#set(player_stats_combined_df['Season'])

In [19]:
player_stats_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373069 entries, 0 to 27739
Data columns (total 26 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Player    373069 non-null  object 
 1   Date      373069 non-null  object 
 2   Team      373069 non-null  object 
 3   Against   373069 non-null  object 
 4   Home      373069 non-null  int64  
 5   MP        311326 non-null  float64
 6   FG        311326 non-null  float64
 7   FGA       311326 non-null  float64
 8   FG%       295932 non-null  float64
 9   3P        311326 non-null  float64
 10  3PA       311326 non-null  float64
 11  3P%       199196 non-null  float64
 12  FT        311326 non-null  float64
 13  FTA       311326 non-null  float64
 14  FT%       179762 non-null  float64
 15  ORB       311326 non-null  float64
 16  DRB       311326 non-null  float64
 17  TRB       311326 non-null  float64
 18  AST       311326 non-null  float64
 19  STL       311326 non-null  float64
 20  BLK  

In [20]:
player_stats_combined_df.describe()

,Home,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
count,373069.000000,311326.000000,311326.000000,311326.000000,295932.000000,311326.00000,311326.000000,199196.000000,311326.000000,311326.000000,...,311326.000000,311326.000000,311326.00000,311326.000000,311326.000000,311326.000000,311326.000000,311326.000000,311326.000000,311276.000000
mean,0.515875,22.840477,3.641630,7.986805,0.441478,0.80294,2.250056,0.325172,1.693896,2.223743,...,1.011412,3.075278,4.08669,2.118628,0.723187,0.464661,1.302291,1.952192,9.780095,-0.010345
std,0.499749,10.922109,3.017081,5.615867,0.236252,1.23164,2.554412,0.307660,2.315300,2.827887,...,1.395258,2.705052,3.50173,2.487667,0.973774,0.856513,1.392476,1.475046,8.025905,10.754541
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-57.000000
25%,0.000000,15.040000,1.000000,4.000000,0.313000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,-7.000000
50%,1.000000,23.450000,3.000000,7.000000,0.444000,0.00000,1.000000,0.333000,1.000000,2.000000,...,1.000000,2.000000,3.00000,1.000000,0.000000,0.000000,1.000000,2.000000,8.000000,0.000000
75%,1.000000,31.460000,5.000000,11.000000,0.571000,1.00000,4.000000,0.500000,3.000000,4.000000,...,2.000000,4.000000,6.00000,3.000000,1.000000,1.000000,2.000000,3.000000,14.000000,7.000000
max,1.000000,64.580000,24.000000,50.000000,1.000000,14.00000,24.000000,1.000000,26.000000,39.000000,...,18.000000,23.000000,31.00000,25.000000,10.000000,12.000000,12.000000,6.000000,70.000000,57.000000


In [33]:
player_stats_combined_df.isnull().sum()

Player          0
Date            0
Team            0
Against         0
Home            0
MP          61743
FG          61743
FGA         61743
FG%         77137
3P          61743
3PA         61743
3P%        173873
FT          61743
FTA         61743
FT%        193307
ORB         61743
DRB         61743
TRB         61743
AST         61743
STL         61743
BLK         61743
TOV         61743
PF          61743
PTS         61743
+/-         61793
Ast/To     111847
dtype: int64

In [34]:
player_stats_combined_df[player_stats_combined_df['3P%'].isnull()]

,Player,Date,Team,Against,Home,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-,Ast/To
3,Zydrunas Ilgauskas,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,30.24,3.0,5.0,0.600,0.0,...,5.0,8.0,1.0,0.0,0.0,4.0,3.0,15.0,7.0,0.25
4,Ben Wallace,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,19.03,1.0,1.0,1.000,0.0,...,4.0,4.0,0.0,0.0,1.0,1.0,2.0,2.0,-1.0,0.00
5,Anderson Varejão,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,26.09,3.0,6.0,0.500,0.0,...,5.0,9.0,1.0,3.0,0.0,2.0,2.0,9.0,-7.0,0.50
9,Lorenzen Wright,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,10.29,1.0,3.0,0.333,0.0,...,2.0,2.0,1.0,1.0,0.0,1.0,4.0,2.0,0.0,1.00
10,Tarence Kinsey,2008-10-28,Cleveland Cavaliers,Boston Celtics,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27715,J.R. Smith,2020-10-09,Los Angeles Lakers,Miami Heat,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27716,Jared Dudley,2020-10-09,Los Angeles Lakers,Miami Heat,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27717,Quinn Cook,2020-10-09,Los Angeles Lakers,Miami Heat,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27730,Bam Adebayo,2020-10-11,Miami Heat,Los Angeles Lakers,1,42.13,10.0,15.0,0.667,0.0,...,9.0,10.0,5.0,0.0,2.0,2.0,2.0,25.0,-21.0,2.50


In [71]:
threeptking = player_stats_combined_df[['Player','Team', 'Against', 'Date', '3PA', 'PTS']][player_stats_combined_df['3P%']==1]
threeptking[threeptking['3PA']>8]

,Player,Team,Against,Date,3PA,PTS
17170,Ben Gordon,Detroit Pistons,Denver Nuggets,2012-03-21,9.0,45.0


In [22]:
print(player_stats_combined_df[['Player','Date','3PA','Against','3P%']][player_stats_combined_df['3PA']>8])

                Player        Date   3PA                 Against    3P%
156     Jamal Crawford  2008-10-29  10.0              Miami Heat  0.400
217       Kevin Martin  2008-10-29  11.0  Minnesota Timberwolves  0.273
347    Stephen Jackson  2008-10-29  12.0     New Orleans Hornets  0.417
456      Al Harrington  2008-10-31  12.0   Golden State Warriors  0.500
457    Stephen Jackson  2008-10-31  11.0   Golden State Warriors  0.364
...                ...         ...   ...                     ...    ...
27647  Markieff Morris  2020-10-04  11.0              Miami Heat  0.455
27654  Duncan Robinson  2020-10-04  10.0      Los Angeles Lakers  0.300
27693      Jae Crowder  2020-10-09   9.0      Los Angeles Lakers  0.222
27695  Duncan Robinson  2020-10-09  13.0      Los Angeles Lakers  0.538
27706     LeBron James  2020-10-09   9.0              Miami Heat  0.667

[8186 rows x 5 columns]


In [23]:
player_stats_combined_df['Ast/To'] = player_stats_combined_df['AST']/player_stats_combined_df['TOV']

In [24]:
player_stats_combined_df = player_stats_combined_df.drop(['GameLink'], axis = 1)

In [25]:
player_stats_combined_df['Date'] = pd.to_datetime(player_stats_combined_df['Date'])

In [26]:
player_stats_combined_df['Date']

0       2008-10-28
1       2008-10-28
2       2008-10-28
3       2008-10-28
4       2008-10-28
           ...    
27735   2020-10-11
27736   2020-10-11
27737   2020-10-11
27738   2020-10-11
27739   2020-10-11
Name: Date, Length: 373069, dtype: datetime64[ns]

In [27]:
#Dataset acquired from kaggle: 
#website - "https://www.kaggle.com/rafaelgreca/nba-games-box-score-since-1949"

cs_team = pd.read_csv('/Users/Shawn/Documents/UNH_ML_SLF/MLassignments/UNH-Repo/DS_Guided_Capstone_2_Repo/virenv/team_stats/2019-2020.csv')

In [28]:
cs_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Columns: 124 entries, gmDate to matchWinner
dtypes: float64(110), int64(4), object(10)
memory usage: 1.1+ MB


In [29]:
print(cs_team.columns.tolist())

['gmDate', 'seasonType', 'season', 'teamWins', 'teamLosses', 'teamAbbr', 'teamLoc', 'teamRslt', 'teamDayOff', 'teamPTS', 'teamAST', 'teamTO', 'teamMin', 'teamSTL', 'teamBLK', 'teamPF', 'teamFGA', 'teamFGM', 'teamFG%', 'team2PA', 'team2PM', 'team2P%', 'team3PA', 'team3PM', 'team3P%', 'teamFTA', 'teamFTM', 'teamFT%', 'teamORB', 'teamDRB', 'teamTRB', 'teamPTS1', 'teamPTS2', 'teamPTS3', 'teamPTS4', 'teamPTS5', 'teamPTS6', 'teamPTS7', 'teamPTS8', 'teamPTS9', 'teamPTS10', 'teamTREB%', 'teamASST%', 'teamTS%', 'teamEFG%', 'teamOREB%', 'teamDREB%', 'teamTO%', 'teamSTL%', 'teamBLK%', 'teamBLKR', 'teamPPS', 'teamFIC', 'teamFIC40', 'teamOrtg', 'teamDrtg', 'teamEDiff', 'teamPlay%', 'teamAR', 'teamPoss', 'teamAST/TO', 'teamPace', 'teamSTL/TO', 'opptWins', 'opptLosses', 'opptAbbr', 'opptLoc', 'opptRslt', 'opptDayOff', 'opptPTS', 'opptAST', 'opptTO', 'opptMin', 'opptSTL', 'opptBLK', 'opptPF', 'opptFGA', 'opptFGM', 'opptFG%', 'oppt2PA', 'oppt2PM', 'oppt2P%', 'oppt3PA', 'oppt3PM', 'oppt3P%', 'opptFTA', 

In [30]:
cs_team.head()

,gmDate,seasonType,season,teamWins,teamLosses,teamAbbr,teamLoc,teamRslt,teamDayOff,teamPTS,...,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,opptPoss,opptPace,matchWinner
0,2019-10-22,Regular,2019-20,0,1,NOP,Away,Loss,0.0,122.0,...,115.2,108.1,7.1,0.41,18.36,1.44,0.44,112.845833,102.2,TOR
1,2019-10-22,Regular,2019-20,0,1,LAL,Away,Loss,0.0,102.0,...,117.5,107.0,10.5,0.50,22.13,1.71,0.57,95.300000,95.3,LAC
2,2019-10-23,Regular,2019-20,0,1,CHI,Away,Loss,0.0,125.0,...,124.0,123.0,1.0,0.46,21.88,1.47,0.16,101.600000,101.6,CHO
3,2019-10-23,Regular,2019-20,1,0,DET,Away,Win,0.0,119.0,...,109.8,118.8,-9.0,0.44,21.77,1.73,0.53,100.200000,100.2,DET
4,2019-10-23,Regular,2019-20,0,1,CLE,Away,Loss,0.0,85.0,...,95.2,86.1,9.1,0.40,20.94,1.85,0.92,98.700000,98.7,ORL
